# <center>Travail d'Étude</center>
## <center>Calcul formel et tracés de graphiques</center>

In [ ]:
from sympy import *
from sympy.abc import p, theta, phi
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


## Les imports<br>
**sympy** est utilisé pour le calcul formel. Pour des raisons de simplicité, toutes les fonctions utilisées dans ce documents sont calculées informatiquement, il a été vérifié par analyse graphique que les résultats des calculs faits à la main correspondent aux résultats déterminés par ordinateur.<br>
**numpy** est prinipalement utilisé pour la gestion des formules une fois converties en fonctions afin de pouvoir les tracer correctement.<br>
**matplotlib** et **mpl_toolkits.mplot3d** servent à tracer les graphes qui nous permettent de visualiser la fidélité pour les différents cas étudiés.

## Les objets mathématiques<br>

In [ ]:
# Déclaration des variable

theta = Symbol('theta')
phi = Symbol('phi')
p = Symbol('p')

# Déclaration des matrices de Pauli
sigmaX = Matrix([[0, 1],
				 [1, 0]])
sigmaY = Matrix([[0, -I],
				 [I, 0]])
sigmaZ = Matrix([[1, 0],
			   	 [0, -1]])
splus = Matrix([[0, 1],
				[0, 0]])
smoins = Matrix([[0, 0],
				 [1, 0]])
# Identité
Id = Matrix([[1,0],
             [0,1]])

# Hadamard
H = (1/sqrt(2))*Matrix([[1, 1],
			            [1, -1]])
# Racine(X)
sX = (1/sqrt(2))*Matrix([[1, -I],
			            [-I, 1]])
# Racine(W)
sW = (1/sqrt(2))*Matrix([[1, -exp(I*pi/4)],
			            [exp(-I*pi/4), 1]])

# rho_0
rhozero = Matrix([[cos(theta / 2) ** 2, exp(I * phi) * sin(theta) / 2],
				  [exp(-I * phi) * sin(theta) / 2, sin(theta / 2) ** 2]])


Les variables *theta*, *phi* et *p* sont déclarées en tant que **Symbol()**. Ce sont des objets de **sympy** qui sont alors utilisables pour du calcul formel.
Il en va de même pour toutes les matrices déclarées comme **Matrix()**.

In [ ]:
def deltap(rho):
	return simplify(trigsimp((1 - p) * (sigmaZ * rho * sigmaZ) + p * (splus * rho * smoins) + p * (smoins * rho * splus)))

def deltap2(rho):
	return (1 - (p/2)) * (sigmaZ * rho * sigmaZ) + (p/2) * (splus * rho * smoins) + (p/2) * (smoins * rho * splus)


Définition de l'application de dépolarisation $\Delta_p$, ainsi que $\dfrac{\Delta_p}{2}$. L'argument $\rho$ attendu est une matrice 2x2 (toutes les matrices traitées ici sont 2x2).<br>
La fonction **simplify()** permet de simplifier autant que possible les expressions obtenues. La fonction **trigsimp()** simplfie également une expression en utilisant les identités trigonométriques.<br>

In [ ]:
def av(M):
    return M*(deltap(rhozero))*((conjugate(M)).T)

def ap(M):
    return deltap(M*rhozero*((conjugate(M)).T))

def avap(M):
    return deltap2(M*(deltap2(rhozero))*((conjugate(M)).T))


Chacune de ces fonctions sert à appliquer du bruits sur les portes, respectivement avant la porte pour **av(M)**, après la porte pour **ap(M)** et avant et après la porte pour **avap(M)**. $M$ étant la matrice à étudier.

In [ ]:
def fid(rho, M):
	temp = rho * (M*rhozero*((conjugate(M)).T))
	return temp[0,0]+temp[1,1]


La fonction **fid(rho, M)** sert à calculer la fidélité. Avec $\rho$ la matrice après application du bruit et $M$ la matrice avant application du bruit.

In [ ]:
fct_id_av = lambdify([theta, phi, p], fid(av(Id), Id), 'numpy')
fct_id_avap = lambdify([theta, phi, p], fid(avap(Id), Id), 'numpy')

fct_H_ap = lambdify([theta, phi, p], fid(ap(H), H), 'numpy')
fct_H_avap = lambdify([theta, phi, p], fid(avap(H), H), 'numpy')

fct_sX_ap = lambdify([theta, phi, p], fid(ap(sX), sX), 'numpy')
fct_sX_avap = lambdify([theta, phi, p], fid(avap(sX), sX), 'numpy')

fct_sW_ap = lambdify([theta, phi, p], fid(ap(sW), sW), 'numpy')
fct_sW_avap = lambdify([theta, phi, p], fid(avap(sW), sW), 'numpy')


Ce bloc sert à calculer convertir les fonction **sympy** en fonction **numpy** afin de pouvoir tracer des graphiques.<br>
Son exécution est un peu longue car les calculs pour les portes de Hadamart, $\sqrt{X}$ et $\sqrt{W}$ sont plus compliqués que les calculs sur les matrices de Pauli et la matrice identité.<br>
Il est volontaire que ce bloc ne calcul pas toutes les fonctions car certaines sont identiques et afin d'optimiser le temps de calcul, nous n'avons pas programmé le calcul pour les fonctions qui se répètent. Les fonctions non calculées sont les suivantes :<br>
1. Les matrices de Pauli, qui donnent des résultats identiques à la matrice identité, que le bruit soit appliqué avant, après ou avant et après la porte.<br>
2. La matrice identité avec un bruit appliqué après la porte, donne un résultat identique à celui obtenu avec le bruit appliqué avant la porte.<br>
3. La porte de Hadamart, qui donne des résultats identiques à la matrice identité quand le bruit est appliqué avant la porte.<br>
4. Les portes $\sqrt{X}$ et $\sqrt{W}$, qui donnent des résultats identiques à la matrice identité lorsque le bruit est appliqué avant la porte.

## Les graphes<br>
À partir d'ici, l'exécution de tous les blocs n'est plus nécessaire, afin de rendre le programme plus lisible, les tracés des graphes sont séparés de la manière suivante :<br>
1. Les graphes sans déphasages
2. Les graphes pour la phase de i
3. Les graphes pour l'état propre
4. Les graphes pour le chat de Schrödinger


In [ ]:
%%capture

fig = plt.figure(figsize=(24, 30))

ax1 = fig.add_subplot(4, 2, 1, projection='3d')
ax2 = fig.add_subplot(4, 2, 2, projection='3d')
ax3 = fig.add_subplot(4, 2, 3, projection='3d')
ax4 = fig.add_subplot(4, 2, 4, projection='3d')
ax5 = fig.add_subplot(4, 2, 5, projection='3d')
ax6 = fig.add_subplot(4, 2, 6, projection='3d')
ax7 = fig.add_subplot(4, 2, 7, projection='3d')
ax8 = fig.add_subplot(4, 2, 8, projection='3d')


Ce bloc crée les 8 subplots qui seront utilisés pour l'affichage. Une seule exécution suffit, le changement de cas se fait plus tard.<br>
**%%capture** empêche Jupyter d'afficher les graphes vides au moment de leur déclaration.

La fonction suivante est créée pour rendre le tracé des graphes moins lourd (évite les gros blocs de texte répétitifs pour les mêmes plots). Elle n'a aucun impact sur les expressions.

In [ ]:
def plot_fid(ax, x, y, z, cas, title):
    ax.clear()
    
    if cas == "cas1" or cas == "cas2":
        ax.set_xlabel("theta")
    elif cas == "cas3" or cas == "cas4":
        ax.set_xlabel("phi")

    ax.set_ylabel("p")  
    ax.set_zlabel("F")
    ax.set_title(title)
    ax.grid()
    
    return ax.plot_surface(x, y, z, cmap='jet')

Pour enregistrer les graphiques, il suffit d'enlever le commentaire de la ligne **fig.savefig("----.jpeg")** dans les blocs appelant la fonction **plot_fid()**.
Le format par défaut est le jpeg, il est possible d'enregistrer en pdf pour avoir des images vectorielles, mais les fichiers deviennent rapidement lourds si les blocs sont exécutés plusieurs fois sans relancer le noyau python3 régulièrement.

### 1. Pas de déphasage
$F (\theta, 0, p)$

In [ ]:
pas = 100
theta = np.linspace(0, np.pi, pas)
p = np.linspace(0, 1, pas)
phi = 0

X, Y = np.meshgrid(theta, p)

Z1 = np.real(fct_id_av(X, phi, Y))
Z2 = np.real(fct_H_ap(X, phi, Y))
Z3 = np.real(fct_sX_ap(X, phi, Y))
Z4 = np.real(fct_sW_ap(X, phi, Y))
Z5 = np.real(fct_id_avap(X, phi, Y))
Z6 = np.real(fct_H_avap(X, phi, Y))
Z7 = np.real(fct_sX_avap(X, phi, Y))
Z8 = np.real(fct_sW_avap(X, phi, Y))


Ce bloc crée des listes pour $\theta$ et $p$, affecte la valeur 0 à $\varphi$ et calcul les éléments nécessaires au tracé des plots 3D.<br>
Le pas peut-être changé, mais le calcul devient rapidement lourd à force de l'augmenter, 100 est une valeur correcte qui nous permet de voir correctement les plots.<br>
Ce bloc est copié et modifié dans chacun des 4 cas en changeant les éléments nécessaires. (voir plus bas)

In [ ]:
plot_fid(ax1, X, Y, Z1, "cas1", "Id avant")
plot_fid(ax2, X, Y, Z2, "cas1", "H après")
plot_fid(ax3, X, Y, Z3, "cas1", "sqrtX après")
plot_fid(ax4, X, Y, Z4, "cas1", "sqrtW après")
plot_fid(ax5, X, Y, Z5, "cas1", "Id avant et après")
plot_fid(ax6, X, Y, Z6, "cas1", "H avant et après")
plot_fid(ax7, X, Y, Z7, "cas1", "sqrtX avant et après")
plot_fid(ax8, X, Y, Z8, "cas1", "sqrtW avant et après")

# fig.savefig("pas_de_dephasage.jpeg")
fig


Ces graphes sont beaux, mais l'interprétation est compliquée. $\theta$ étant responsable de l'aspect sinusoïdal, on peut tracer les plots uniquement en fonction de $\theta$ et superposer les résultats pour différentes valeurs de $p$ afin d'étudier son effet.<br>
Nous allons donc tracer des graphes 2D pour 6 valeurs de $p$ (0,0 ; 0,2 ; 0,4 ; 0,6 ; 0,8 ; 1,0).

Même chose que plus haut, la fonction suivante est appelé pour tracer les graphes 2D sans avoir à répéter les mêmes lignes de code.

In [ ]:
def plot_2d(ax, x, fct, const, cas, title):
    ax.clear()
    if cas == "cas1" or cas == "cas2":
        ax.set_xlabel("theta")
    elif cas == "cas3" or cas == "cas4":
        ax.set_xlabel("phi")
    
    ax.set_ylabel("F")
    ax.set_ylim([-0.2, 1.2])
    ax.set_title(title)
    
    for i in range(0, 11, 2):
        if fct == "id_av":
            ax.plot(x, np.real(fct_id_av(x, const, i / 10)), label="p = %1.1f"%(i / 10))
        elif fct == "H_ap":
            ax.plot(x, np.real(fct_H_ap(x, const, i / 10)), label="p = %1.1f"%(i / 10))
        elif fct == "sX_ap":
            ax.plot(x, np.real(fct_sX_ap(x, const, i / 10)), label="p = %1.1f"%(i / 10))
        elif fct == "sW_ap":
            ax.plot(x, np.real(fct_sW_ap(x, const, i / 10)), label="p = %1.1f"%(i / 10))
        elif fct == "id_avap":
            ax.plot(x, np.real(fct_id_avap(x, const, i / 10)), label="p = %1.1f"%(i / 10))
        elif fct == "H_avap":
            ax.plot(x, np.real(fct_H_avap(x, const, i / 10)), label="p = %1.1f"%(i / 10))
        elif fct == "sX_avap":
            ax.plot(x, np.real(fct_sX_avap(x, const, i / 10)), label="p = %1.1f"%(i / 10))
        elif fct == "sW_avap":
            ax.plot(x, np.real(fct_sW_avap(x, const, i / 10)), label="p = %1.1f"%(i / 10))
    ax.grid()
    ax.legend()


In [ ]:
%%capture

fig2 = plt.figure(figsize=(24, 30))

ax11 = fig2.add_subplot(4, 2, 1)
ax12 = fig2.add_subplot(4, 2, 2)
ax13 = fig2.add_subplot(4, 2, 3)
ax14 = fig2.add_subplot(4, 2, 4)
ax15 = fig2.add_subplot(4, 2, 5)
ax16 = fig2.add_subplot(4, 2, 6)
ax17 = fig2.add_subplot(4, 2, 7)
ax18 = fig2.add_subplot(4, 2, 8)


In [ ]:
phi = 0
pas = 100
theta = np.linspace(0, np.pi, pas)

plot_2d(ax11, theta, "id_av", phi, "cas1", "Id avant")
plot_2d(ax12, theta, "H_ap", phi, "cas1", "H après")
plot_2d(ax13, theta, "sX_ap", phi, "cas1", "sqrtX après")
plot_2d(ax14, theta, "sW_ap", phi, "cas1", "sqrtW après")
plot_2d(ax15, theta, "id_avap", phi, "cas1", "Id avant")
plot_2d(ax16, theta, "H_avap", phi, "cas1", "H avant et après")
plot_2d(ax17, theta, "sX_avap", phi, "cas1", "sqrtX avant et après")
plot_2d(ax18, theta, "sW_avap", phi, "cas1", "sqrtX avant et après")
fig2


# 2. Phase de i
$F (\theta, \pi, p)$

In [ ]:
pas = 100
theta = np.linspace(0, np.pi, pas)
p = np.linspace(0, 1, pas)
phi = np.pi

X, Y = np.meshgrid(theta, p)

Z1 = np.real(fct_id_av(X, phi, Y))
Z2 = np.real(fct_H_ap(X, phi, Y))
Z3 = np.real(fct_sX_ap(X, phi, Y))
Z4 = np.real(fct_sW_ap(X, phi, Y))
Z5 = np.real(fct_id_avap(X, phi, Y))
Z6 = np.real(fct_H_avap(X, phi, Y))
Z7 = np.real(fct_sX_avap(X, phi, Y))
Z8 = np.real(fct_sW_avap(X, phi, Y))


In [ ]:
plot_fid(ax1, X, Y, Z1, "cas2", "Id avant")
plot_fid(ax2, X, Y, Z2, "cas2", "H après")
plot_fid(ax3, X, Y, Z3, "cas2", "sqrtX après")
plot_fid(ax4, X, Y, Z4, "cas2", "sqrtW après")
plot_fid(ax5, X, Y, Z5, "cas2", "Id avant et après")
plot_fid(ax6, X, Y, Z6, "cas2", "H avant et après")
plot_fid(ax7, X, Y, Z7, "cas2", "sqrtX avant et après")
plot_fid(ax8, X, Y, Z8, "cas2", "sqrtW avant et après")

# fig.savefig("phase_de_i.jpeg")
fig


In [ ]:
phi = np.pi / 2
pas = 100
theta = np.linspace(0, np.pi, pas)

plot_2d(ax11, theta, "id_av", phi, "cas2", "Id avant")
plot_2d(ax12, theta, "H_ap", phi, "cas2", "H après")
plot_2d(ax13, theta, "sX_ap", phi, "cas2", "sqrtX après")
plot_2d(ax14, theta, "sW_ap", phi, "cas2", "sqrtW après")
plot_2d(ax15, theta, "id_avap", phi, "cas2", "Id avant")
plot_2d(ax16, theta, "H_avap", phi, "cas2", "H avant et après")
plot_2d(ax17, theta, "sX_avap", phi, "cas2", "sqrtX avant et après")
plot_2d(ax18, theta, "sW_avap", phi, "cas2", "sqrtX avant et après")
fig2


### 3. État propre
$F (\pi, \varphi, p)$

In [ ]:
pas = 100
phi = np.linspace(0, 2*np.pi, pas)
p = np.linspace(0, 1, pas)
theta = np.pi

X, Y = np.meshgrid(phi, p)

Z1 = np.real(fct_id_av(theta, X, Y))
Z2 = np.real(fct_H_ap(theta, X, Y))
Z3 = np.real(fct_sX_ap(theta, X, Y))
Z4 = np.real(fct_sW_ap(theta, X, Y))
Z5 = np.real(fct_id_avap(theta, X, Y))
Z6 = np.real(fct_H_avap(theta, X, Y))
Z7 = np.real(fct_sX_avap(theta, X, Y))
Z8 = np.real(fct_sW_avap(theta, X, Y))


In [ ]:
plot_fid(ax1, X, Y, Z1, "cas3", "Id avant")
plot_fid(ax2, X, Y, Z2, "cas3", "H après")
plot_fid(ax3, X, Y, Z3, "cas3", "sqrtX après")
plot_fid(ax4, X, Y, Z4, "cas3", "sqrtW après")
plot_fid(ax5, X, Y, Z5, "cas3", "Id avant et après")
plot_fid(ax6, X, Y, Z6, "cas3", "H avant et après")
plot_fid(ax7, X, Y, Z7, "cas3", "sqrtX avant et après")
plot_fid(ax8, X, Y, Z8, "cas3", "sqrtW avant et après")

# fig.savefig("etat_propre.jpeg")
fig


### 3.5 États propres - Partie imaginaire<br>
Les graphes pour les états propres semblent trop "simples". Nous avons donc tracé la partie imaginaire de la fidélité, cela donne des graphiques plutôt surprenants.

In [ ]:
pas = 100
phi = np.linspace(0, 2*np.pi, pas)
p = np.linspace(0, 1, pas)
theta = np.pi

X, Y = np.meshgrid(phi, p)

Z1 = np.imag(fct_id_av(theta, X, Y))
Z2 = np.imag(fct_H_ap(theta, X, Y))
Z3 = np.imag(fct_sX_ap(theta, X, Y))
Z4 = np.imag(fct_sW_ap(theta, X, Y))
Z5 = np.imag(fct_id_avap(theta, X, Y))
Z6 = np.imag(fct_H_avap(theta, X, Y))
Z7 = np.imag(fct_sX_avap(theta, X, Y))
Z8 = np.imag(fct_sW_avap(theta, X, Y))

In [ ]:
plot_fid(ax1, X, Y, Z1, "cas3", "Id avant")
plot_fid(ax2, X, Y, Z2, "cas3", "H après")
plot_fid(ax3, X, Y, Z3, "cas3", "sqrtX après")
plot_fid(ax4, X, Y, Z4, "cas3", "sqrtW après")
plot_fid(ax5, X, Y, Z5, "cas3", "Id avant et après")
plot_fid(ax6, X, Y, Z6, "cas3", "H avant et après")
plot_fid(ax7, X, Y, Z7, "cas3", "sqrtX avant et après")
plot_fid(ax8, X, Y, Z8, "cas3", "sqrtW avant et après")

# fig.savefig("etat_propre_complexe.jpeg")
fig


### 4. Chat de Schrödinger

In [ ]:
pas = 100
phi = np.linspace(0, 2*np.pi, pas)
p = np.linspace(0, 1, pas)
theta = np.pi / 2

X, Y = np.meshgrid(phi, p)

Z1 = np.real(fct_id_av(theta, X, Y))
Z2 = np.real(fct_H_ap(theta, X, Y))
Z3 = np.real(fct_sX_ap(theta, X, Y))
Z4 = np.real(fct_sW_ap(theta, X, Y))
Z5 = np.real(fct_id_avap(theta, X, Y))
Z6 = np.real(fct_H_avap(theta, X, Y))
Z7 = np.real(fct_sX_avap(theta, X, Y))
Z8 = np.real(fct_sW_avap(theta, X, Y))

In [ ]:
plot_fid(ax1, X, Y, Z1, "cas4", "Id avant")
plot_fid(ax2, X, Y, Z2, "cas4", "H après")
plot_fid(ax3, X, Y, Z3, "cas4", "sqrtX après")
plot_fid(ax4, X, Y, Z4, "cas4", "sqrtW après")
plot_fid(ax5, X, Y, Z5, "cas4", "Id avant et après")
plot_fid(ax6, X, Y, Z6, "cas4", "H avant et après")
plot_fid(ax7, X, Y, Z7, "cas4", "sqrtX avant et après")
plot_fid(ax8, X, Y, Z8, "cas4", "sqrtW avant et après")

fig.savefig("chat_de_schrodinger.jpeg")
fig
